In [11]:
import pandas as pd
import datetime
from cdapython import tables, columns, column_values, fetch_rows, summary_counts

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

ct = datetime.datetime.now()
print("current time:-", ct)








current time:- 2024-06-04 13:14:53.921608


In [12]:
sub = fetch_rows(table='subject', add_columns=['diagnosis_id'])

sub['subject_id'].nunique()

86359

In [13]:
research = fetch_rows(table='researchsubject', add_columns=['diagnosis_id'])

research.head()

,researchsubject_id,member_of_research_project,primary_diagnosis_condition,primary_diagnosis_site,diagnosis_id
2445,APOLLO-LUAD.AP-27JZ,APOLLO-LUAD,Adenomas and Adenocarcinomas,Bronchus and lung,APOLLO-LUAD.AP-27JZ.AP-27JZ-Diagnosis
2446,APOLLO-LUAD.AP-2E8Q,APOLLO-LUAD,Adenomas and Adenocarcinomas,Bronchus and lung,APOLLO-LUAD.AP-2E8Q.AP-2E8Q-Diagnosis
2447,APOLLO-LUAD.AP-2XNK,APOLLO-LUAD,Adenomas and Adenocarcinomas,Bronchus and lung,APOLLO-LUAD.AP-2XNK.AP-2XNK-Diagnosis
2448,APOLLO-LUAD.AP-3336,APOLLO-LUAD,Adenomas and Adenocarcinomas,Bronchus and lung,APOLLO-LUAD.AP-3336.AP-3336-Diagnosis
2449,APOLLO-LUAD.AP-352Q,APOLLO-LUAD,Adenomas and Adenocarcinomas,Bronchus and lung,APOLLO-LUAD.AP-352Q.AP-352Q-Diagnosis


In [9]:
diag = fetch_rows(table='diagnosis', provenance=True)

diag['diagnosis_data_source_id_concat'] = diag.groupby(['diagnosis_id','diagnosis_data_source'])['diagnosis_data_source_id'].transform(lambda x: ','.join(x))
diag = diag.drop(columns=['diagnosis_data_source_id'], axis=1)
diag = diag.drop_duplicates()

diag.head()

,diagnosis_id,age_at_diagnosis,grade,method_of_diagnosis,morphology,primary_diagnosis,stage,diagnosis_data_source,diagnosis_data_source_id_concat
0,APOLLO-LUAD.AP-27JZ.AP-27JZ-Diagnosis,68,G3,,Acinar cell carcinoma,Acinar Cell Carcinoma,,GDC,case.diagnoses.diagnosis_id:06c35117-f62f-4470...
2,APOLLO-LUAD.AP-2E8Q.AP-2E8Q-Diagnosis,47,G3,,Acinar cell carcinoma,Acinar Cell Carcinoma,,GDC,case.diagnoses.diagnosis_id:2d5f86b2-f3dd-4554...
4,APOLLO-LUAD.AP-2XNK.AP-2XNK-Diagnosis,50,G3,,"Papillary Adenocarcinoma, NOS",Papillary Adenocarcinoma,,GDC,case.diagnoses.diagnosis_id:b7e1b994-2d54-4719...
6,APOLLO-LUAD.AP-3336.AP-3336-Diagnosis,79,G2,,"Papillary Adenocarcinoma, NOS",Papillary Adenocarcinoma,,GDC,case.diagnoses.diagnosis_id:fcb58ffe-73e6-4a6c...
8,APOLLO-LUAD.AP-352Q.AP-352Q-Diagnosis,66,G3,,"Solid carcinoma, NOS",Solid Carcinoma,,GDC,case.diagnoses.diagnosis_id:bdd7269a-651d-4117...


In [18]:
diag_GDC = diag[diag['diagnosis_data_source']=='GDC']
diag_GDC
diag_GDC_sort = diag_GDC['stage'].sort_values(ascending=False)
diag_GDC_sort

0          
125707     
125691     
125693     
125695     
         ..
29198      
29200      
29202      
29204      
154895     
Name: stage, Length: 43794, dtype: object

In [2]:
from cdapython import tables, columns, column_values, fetch_rows, summary_counts

#sub = fetch_rows(table='subject', provenance=True)
subj_gdc = fetch_rows(table='subject', data_source = ["GDC"])

subj_gdc[['subject_id_proj','subject_id_tail']] = subj_gdc['subject_id'].str.rsplit('.',n=1,expand=True)
subj_gdc = subj_gdc[['subject_id','subject_id_proj','subject_id_tail']]
subj_gdc['subject_id_tail'].nunique()
subj_list = subj_gdc['subject_id_tail'].unique().tolist()


In [8]:
import requests
import pandas as pd
#import json
#from io import BytesIO
from io import StringIO

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

'''
fields = [
    "submitter_id",
    "case_id",
    "primary_site",
    "disease_type",
    "diagnoses.vital_status"
    ]
'''

fields = [
    "submitter_id",
    "diagnoses.diagnosis_id",
    "diagnoses.ajcc_pathologic_stage"
    ]

fields = ",".join(fields)
print(fields)

case_endpt = "https://api.gdc.cancer.gov/cases"
filters = {"op":"and","content":[{"op":"in","content":{"field":"submitter_id","value":subj_list}}]}
params = {
    "filters": filters,
    "fields": fields,
    "format": "TSV",
    "size": "50000"
    }

# The parameters are passed to 'json' rather than 'params' in this case
response_b = requests.post(case_endpt, headers = {"Content-Type": "application/json"}, json = params)
response_b_df = pd.read_csv(StringIO(response_b.text), sep="\t")
response_b_df.head()

submitter_id,diagnoses.diagnosis_id,diagnoses.ajcc_pathologic_stage


,diagnoses.0.ajcc_pathologic_stage,diagnoses.0.diagnosis_id,diagnoses.1.ajcc_pathologic_stage,diagnoses.1.diagnosis_id,diagnoses.2.ajcc_pathologic_stage,diagnoses.2.diagnosis_id,diagnoses.3.diagnosis_id,id,submitter_id
0,NaN,7b142a98-5022-43e9-bcbb-647d3b28b308,NaN,NaN,NaN,NaN,NaN,69eced5b-1e76-45c9-bc9c-2aa71a921c57,HCM-BROD-0011-C71
1,Stage III,fa6d5c8a-3d61-49e3-805e-e14b58514a7e,NaN,NaN,NaN,NaN,NaN,e3b32485-b204-43a7-93a5-601408fcdf96,HCM-CSHL-0074-C25
2,NaN,a7f019db-c623-4d56-94e9-102cc4574f88,NaN,NaN,NaN,NaN,NaN,4829dd8c-5445-41b3-ae37-bbcc333e8c9e,HCM-BROD-0001-C18
3,NaN,75ba645e-7756-4208-825b-67cff507ac69,NaN,NaN,NaN,NaN,NaN,d420e653-3fb2-432b-9e81-81232a80264d,HCM-BROD-0210-C71
4,Stage I,000db971-fd42-4030-ac42-e606531ec9d0,NaN,NaN,NaN,NaN,NaN,bfe15f44-e1dd-46ed-b429-908822d0a781,HCM-CSHL-0056-C18


In [9]:
response_b_df.value_counts('diagnoses.0.ajcc_pathologic_stage')

diagnoses.0.ajcc_pathologic_stage
Stage I         1674
Stage IIA       1009
Stage III        913
Stage IIB        899
Stage II         735
Stage IV         702
Stage IIIA       642
Stage IB         555
Stage IA         516
Stage IVA        382
Not Reported     350
Stage IIIB       345
Stage IIIC       280
Stage IIC         68
Stage IS          50
Stage IVB         34
Stage X           15
Stage 0            9
Stage IA3          8
Stage IVC          7
Stage IIIC1        6
Unknown            5
Stage IA2          4
Stage IA1          2
Stage IIIA2        1
Stage IIIC2        1
Stage Tis          1
Name: count, dtype: int64

In [51]:
import requests
import json
import pandas as pd
from io import BytesIO
from io import StringIO

# Sample bytes data
bytes_data = b'Name,Age,Occupation\nJohn,25,Engineer\nAlice,30,Doctor\nBob,28,Artist'
 
# Convert bytes to string and then to DataFrame
data_str = bytes_data.decode('utf-8')
df_method1 = pd.read_csv(StringIO(data_str))
 
# Display the DataFrame
print(df_method1)

    Name  Age Occupation
0   John   25   Engineer
1  Alice   30     Doctor
2    Bob   28     Artist


In [70]:
import requests
import json
import pandas as pd
from io import BytesIO

fields = [
    "submitter_id",
    "case_id",
    "primary_site",
    "disease_type",
    "diagnoses.vital_status"
    ]

fields = ",".join(fields)
print(fields)

cases_endpt = "https://api.gdc.cancer.gov/cases"

filters = {
    "op": "in",
    "content":{
        "field": "primary_site",
        "value": ["Kidney"]
        }
    }

# With a GET request, the filters parameter needs to be converted
# from a dictionary to JSON-formatted string

params = {
    "filters": json.dumps(filters),
    "fields": fields,
    "format": "TSV",
    "size": "1000"
    }

response = requests.get(cases_endpt, params = params)

# OUTPUT METHOD 1: Write to a file.
file = open("filtered_query.tsv", "w")
file.write(response.text)
file.close()

#parsed = json.loads(response.content)
#print(parsed)
#df = pd.read_csv(BytesIO(response.content))

content = response.content.decode('utf-8')
#content_repl = content.replace(b'\r', b'')
#print(content_repl)
#data = io.StringIO(content)
#data = content.replace("\\r\\n", "")
print(data)
df = pd.read_csv(StringIO(data),sep='\t')
#print(df)
#print(content)
#d = response.content
#print(d)
#df = pd.read_csv(io.StringIO(d.decode('utf-8')),sep='\r\n')
#df = d.as_df()

#print(df)

submitter_id,case_id,primary_site,disease_type,diagnoses.vital_status
case_id	disease_type	id	primary_site	submitter_id
46b1210c-4042-4c67-9e68-06e960f201f2	Complex Mixed and Stromal Neoplasms	46b1210c-4042-4c67-9e68-06e960f201f2	Kidney	HCM-BROD-0051-C64
819b41c2-1aa4-4a98-959f-096fd3df1446	Complex Mixed and Stromal Neoplasms	819b41c2-1aa4-4a98-959f-096fd3df1446	Kidney	HCM-BROD-0041-C64
54c9850c-9b67-42ea-a04b-16a88f786dfa	Adenomas and Adenocarcinomas	54c9850c-9b67-42ea-a04b-16a88f786dfa	Kidney	TCGA-BP-5201
1cdb170b-9765-45ee-9e3e-388d0a419746	Adenomas and Adenocarcinomas	1cdb170b-9765-45ee-9e3e-388d0a419746	Kidney	TCGA-BP-4158
9a404208-c3d3-4e6b-b511-1a4ccf595c18	Adenomas and Adenocarcinomas	9a404208-c3d3-4e6b-b511-1a4ccf595c18	Kidney	TCGA-BP-4782
6b1ff32f-75a4-44aa-9da4-b6872e179c9f	Adenomas and Adenocarcinomas	6b1ff32f-75a4-44aa-9da4-b6872e179c9f	Kidney	TCGA-B0-5107
6b79d20c-d5bc-4580-837e-b36dfd0c199c	Adenomas and Adenocarcinomas	6b79d20c-d5bc-4580-837e-b36dfd0c199c	Kidney	TCGA-CZ-